In [ ]:
from project_heart.lv import LV
import numpy as np
import pyvista as pv
pv.set_jupyter_backend("pythreejs")

import os
from pathlib import Path

from project_heart.enums import *

In [ ]:
from project_heart.examples import get_lv_ideal
lv = get_lv_ideal("C:/Users/igorp/Downloads/77.81_67.13_157_77.81_67.13_19.07.2022_LVRNN_IDEAL_v4_1.xplt", save_spk_dict=True)

In [ ]:
cy_stress = lv.convert_to_cylindrical_coordinates("stress")
cy_strain = lv.convert_to_cylindrical_coordinates("Lagrange strain")

In [ ]:
cy_stress.shape

In [ ]:
def to_csv(data, file):
    a = np.swapaxes(data, 0,1).reshape((-1,6*61))
    cells = np.arange(1,len(a)+1, dtype=np.int32).reshape(-1,1)
    np.savetxt(file, np.hstack((cells, a)), delimiter=",")

In [ ]:
to_csv(cy_stress, "cy_stress.csv")
to_csv(cy_strain, "cy_strain.csv")

In [ ]:
def to_von_mises(data):
    """Computes the 'effective stress' based on von mises stress.

    Args:
        data (_type_): _description_

    Returns:
        _type_: _description_
    """
    x = data[:, 0]
    y = data[:, 1]
    z = data[:, 2]
    xy = data[:, 3]
    yz = data[:, 4]
    xz = data[:, 5]
    
    vm = x**2 + y**2 + z**2
    vm -= x*y + y*z + x*z
    vm += 3*(xy**2 + yz**2 + xz**2)
    return np.sqrt(vm)

In [ ]:
vm = [to_von_mises(d) for d in cy_stress]

In [ ]:
vm = np.asarray(vm)
vmt = vm.T

In [ ]:
cells = np.arange(1,len(vmt)+1).reshape(-1,1)

In [ ]:
np.savetxt("cy_stress.csv", np.hstack((cells, vmt)), delimiter=",")